**Анализ метода оптимизации многомерных массивов на основе малоранговых тензорных аппроксимаций (TT-метод).**

---

In [1]:
import numpy as np
from qttopt import QTTOpt
import teneva
from time import perf_counter as tpc
np.random.seed(42)

## Построение целевого тензора

Построим явный тензор значений $d$-мерной модельной функции на равномерной сетке из $n$ узлов $Y_{full\_real} \in \mathbb{R}^{n \times n \times \ldots \times n}$:

In [2]:
d = 6
n = 16
name = 'Rosenbrock'
f = teneva.func_demo_all(d, names=[name])[0]
Y_full_real = f.get_f_ind(teneva.grid_flat([n]*d)).reshape([n]*d, order='F')

print(f'Number of elements : {np.size(Y_full_real):-7.1e}')

Number of elements : 1.7e+07


Проверим, на всякий случай, что мы построили правильный тензор:

In [3]:
i = [3, 2, 3, 8, 5, 10]    # Мульти-индекс для проверки
y1 = f.get_f_ind(i)        # Непосредственно значение функции
y2 = Y_full_real[tuple(i)] # Значение тензора
print(y1)
print(y2)

3196.7185927525265
3196.7185927525265


Вычислим точное минимальное и максимальное значение тензора (**заметим, что эти значения очень сильно отличаются от реальных экстремумов функции** ввиду использования недостаточно густой сетки!):

In [4]:
y_min_real = np.min(Y_full_real)
y_max_real = np.max(Y_full_real)

print(f'Min / max value         : {y_min_real:-12.4e} / {y_max_real:-12.4e}')
print(f'* Real function minimum : {f.y_min:-12.4e}')

Min / max value         :   3.0487e-01 /   1.9530e+04
* Real function minimum :   0.0000e+00


## Построение TT-аппроксимации полного тензора посредством TT-SVD

Построим TT-представление тензора посредством TT-SVD:

In [100]:
Y = teneva.svd(Y_full_real, 1.E-6)  # TT-SVD
Y = teneva.truncate(Y, 1.E-6)       # Truncation of result

print(f'TT-rank of res : {teneva.erank(Y):-10.1f}\n')
teneva.show(Y)

TT-rank of res :        3.0

 16 16 16 16 16 16 
 / \/ \/ \/ \/ \/ \
 1  3  3  3  3  3  1 



Проверим точность построенного TT-приближения, переведя построенный TT-тензор в полный формат (замеряем относительную норму разницы тензоров, а также абсолютные ошибки для минимального и максимального значения тензора):

In [101]:
Y_full_appr = teneva.full(Y)
e_norm = teneva.accuracy(Y_full_real, Y_full_appr)
e_min = abs(y_min_real - np.min(Y_full_appr))
e_max = abs(y_max_real - np.max(Y_full_appr))

print(f'Error norm      : {e_norm:-7.1e}')
print(f'Error min value : {e_min:-7.1e}')
print(f'Error max value : {e_max:-7.1e}')

Error norm      : 7.6e-01
Error min value : 3.0e-01
Error max value : 3.0e-01


Попробуем найти максимум в TT-тензоре, используя maxvol:

In [102]:
def _ones(k, m=1):
    return np.ones((k, m), dtype=int)


def _reshape(A, n):
    return np.reshape(A, n, order='F')


def _maxvol(A, tau=1.1, tau0=1.05, k0=100):
    n, r = A.shape
    if n <= r:
        I = np.arange(n, dtype=int)
        B = np.eye(n, dtype=float)
    else:
        I, B = teneva.maxvol(A, tau0, k0)
    return I, B


def _iter(Z, Ig, I=None, l2r=True):
    r1, n, r2 = Z.shape
    Z = _reshape(Z, (r1 * n, r2)) if l2r else _reshape(Z, (r1, n * r2)).T

    Q, R = np.linalg.qr(Z)
    ind, B = _maxvol(Q)

    G = B if l2r else B.T
    G = _reshape(G, (r1, n, -1)) if l2r else _reshape(G, (-1, n, r2))

    R = Q[ind, :] @ R
    R = R if l2r else R.T

    Ig = Ig.reshape((-1, 1))
    I_new = np.kron(Ig, _ones(r1)) if l2r else np.kron(_ones(r2), Ig)
    if I is not None:
        I_old = np.kron(_ones(n), I) if l2r else np.kron(I, _ones(n))
        I_new = np.hstack((I_old, I_new)) if l2r else np.hstack((I_new, I_old))
    I_new = I_new[ind, :]

    return G, R, I_new


def _func(f, Ig, Ir, Ic):
    n = Ig.shape[0]
    r1 = Ir.shape[0] if Ir is not None else 1
    r2 = Ic.shape[0] if Ic is not None else 1

    I = np.kron(np.kron(_ones(r2), Ig), _ones(r1))
    if Ir is not None:
        Ir_ = np.kron(_ones(n * r2), Ir)
        I = np.hstack((Ir_, I))
    if Ic is not None:
        Ic_ = np.kron(Ic, _ones(r1 * n))
        I = np.hstack((I, Ic_))

    y = np.array([f(i) for i in I])
    return _reshape(y, (r1, n, r2))

    
def opt_max(Y):
    Y = teneva.copy(Y)
    d = len(Y)
    n = teneva.shape(Y)
    
    Ig = [_reshape(np.arange(k, dtype=int), (-1, 1)) for k in n]
    Ir = [None for i in range(d+1)]
    Ic = [None for i in range(d+1)]
    
    R = np.ones((1, 1))
    for i in range(d):
        G = np.tensordot(R, Y[i], 1)
        n = G.shape[1]
        Y[i], R, Ir[i+1] = _iter(G, Ig[i], Ir[i], l2r=True)
    Y[d-1] = np.tensordot(Y[d-1], R, 1)

    R = np.ones((1, 1))
    I = None
    for i in range(d-1, -1, -1):
        G = np.tensordot(Y[i], R, 1)
        n = G.shape[1]
        Y[i], R, Ic[i] = _iter(G, Ig[i], Ic[i+1], l2r=False)
    Y[0] = np.tensordot(R, Y[0], 1)
    
    get = teneva.getter(Y)
    y_max = None
    for i in range(d):
        Z = _func(get, Ig[i], Ir[i], Ic[i+1])
        y_max_curr = abs(np.max(Z))
        if y_max is None or y_max_curr > abs(y_max):
            y_max = y_max_curr
            
    return y_max

In [103]:
y_max = opt_max(Y)
e_max = abs(y_max_real - y_max)

print(f'Error max value : {e_max:-7.1e}')

Error max value : 2.8e+01


In [95]:
Y_sub = teneva.add(y_max, teneva.mul(Y, -1.))
Y_sub = teneva.truncate(Y_sub, 1.E-6)

Y_full_appr = teneva.full(Y_sub) #+ y_max
e_norm = teneva.accuracy(Y_full_real, Y_full_appr)
e_min = abs(y_min_real - np.min(Y_full_appr))
e_max = abs(y_max_real - np.max(Y_full_appr))

print(f'Error norm      : {e_norm:-7.1e}')
print(f'Error min value : {e_min:-7.1e}')
print(f'Error max value : {e_max:-7.1e}')

Error norm      : 7.6e-01
Error min value : 3.0e-01
Error max value : 3.0e-01


In [96]:
np.max(Y_full_appr)

19529.326264463547

In [97]:
np.min(Y_full_appr)

-2.0161613993988348e-11

In [98]:
opt_max(Y_sub)

19501.29844848573

In [99]:
y_min = -1 * opt_max(Y_sub) + y_max
e_max = abs(y_min_real - y_min)

print(f'Error min value : {e_max:-7.1e}')

Error min value : 2.8e+01


## Построение TT-аппроксимации посредством TT-CROSS

Теперь построим приближение тензора в TT-формате, используя метод TT-CROSS (для наглядности, будем осуществлять семплирование из построенного точного тензора в полном формате):

In [61]:
def func(I):
    return np.array([Y_full_real[tuple(i)] for i in I])

In [62]:
t = tpc()
info, cache = {}, {}
Y = teneva.rand([n]*d, r=1)
Y = teneva.cross(func, Y, m=1.E+4, dr_min=1, dr_max=2, info=info, cache=cache)
Y = teneva.truncate(Y, 1.E-6)
t = tpc() - t

print(f'Build time     : {t:-10.2f}')
print(f'Evals func     : {info["m"]:-10d}')
print(f'Cache uses     : {info["m_cache"]:-10d}')
print(f'Iter accuracy  : {info["e"]:-10.2e}')
print(f'Sweep number   : {info["nswp"]:-10d}')
print(f'Stop condition : {info["stop"]:>10}')
print(f'TT-rank of res : {teneva.erank(Y):-10.1f}')

Build time     :       0.16
Evals func     :       9232
Cache uses     :       7344
Iter accuracy  :   1.11e+00
Sweep number   :          3
Stop condition :          m
TT-rank of res :        3.0


Проверим точность построенного TT-приближения, переведя построенный TT-тензор в полный формат (замеряем относительную норму разницы тензоров, а также абсолютные ошибки для минимального и максимального значения тензора):

In [63]:
Y_full_appr = teneva.full(Y)
e_norm = teneva.accuracy(Y_full_real, Y_full_appr)
e_min = abs(y_min_real - np.min(Y_full_appr))
e_max = abs(y_max_real - np.max(Y_full_appr))

print(f'Error norm      : {e_norm:-7.1e}')
print(f'Error min value : {e_min:-7.1e}')
print(f'Error max value : {e_max:-7.1e}')

Error norm      : 6.3e-15
Error min value : 8.3e-11
Error max value : 1.7e-10


Найдем минимальное и максимальное значения в кеше (полученном при работе TT-cross) и сравним с точными значениями:

In [64]:
y_min = np.min(list(cache.values()))
y_max = np.max(list(cache.values()))

e_min = abs(y_min_real - y_min)
e_max = abs(y_max_real - y_max)

print(f'Error min value : {e_min:-7.1e}')
print(f'Error max value : {e_max:-7.1e}')

Error min value : 1.4e+02
Error max value : 0.0e+00


**Наблюдение**. Как можно видеть, построенный посредством TT-cross TT-тензор точно приближает и минимальное, и максимальное значения исходного тензора. Однако в кеше запрошенных значений оказывается только хорошее приближение для максимума тензора (что находится в соответствии с теоремой "о максволе").

В то же время, мы можем получить хорошее приближение для максимума, используя максвол по тензору:

In [65]:
y_max = opt_max(Y)
e_max = abs(y_max_real - y_max)

print(f'Error max value : {e_max:-7.1e}')

Error max value : 1.5e-10


## XXX

In [66]:
Y_sub = teneva.add(y_max, teneva.mul(Y, -1))
Y_sub = teneva.truncate(Y_sub, 1.E-6)

Y_full_appr = y_max - teneva.full(Y_sub)
e_norm = teneva.accuracy(Y_full_real, Y_full_appr)
e_min = abs(y_min_real - np.min(Y_full_appr))
e_max = abs(y_max_real - np.max(Y_full_appr))

print(f'Error norm      : {e_norm:-7.1e}')
print(f'Error min value : {e_min:-7.1e}')
print(f'Error max value : {e_max:-7.1e}')

Error norm      : 6.8e-15
Error min value : 6.4e-11
Error max value : 1.6e-10


In [60]:
y_min = y_max - opt_max(Y_sub)
e_max = abs(y_min_real - y_min)

print(f'Error min value : {e_max:-7.1e}')

Error min value : 2.8e+01


## Второй шаг с TT-CROSS

Вычтем из целевой функции найденное (с высокой точностью) максимальное значение и повторно запустим метод TT-cross. Мы ожидаем, что теперь в кеше значений окажется хорошее приближение к минимуму целевого тензора.

In [39]:
y_max_ref = y_max

In [40]:
def func(I):
    return np.array([Y_full_real[tuple(i)] - y_max_ref for i in I])

In [41]:
t = tpc()
info, cache = {}, {}
Y = teneva.rand([n]*d, r=1)
Y = teneva.cross(func, Y, m=1.E+4, dr_min=1, dr_max=2, info=info, cache=cache)
Y = teneva.truncate(Y, 1.E-6)
t = tpc() - t

print(f'Build time     : {t:-10.2f}')
print(f'Evals func     : {info["m"]:-10d}')
print(f'Cache uses     : {info["m_cache"]:-10d}')
print(f'Iter accuracy  : {info["e"]:-10.2e}')
print(f'Sweep number   : {info["nswp"]:-10d}')
print(f'Stop condition : {info["stop"]:>10}')
print(f'TT-rank of res : {teneva.erank(Y):-10.1f}')

Build time     :       0.13
Evals func     :       9048
Cache uses     :       5784
Iter accuracy  :   1.57e-01
Sweep number   :          3
Stop condition :          m
TT-rank of res :        7.8


In [42]:
Y_full_appr = teneva.full(Y) + y_max_ref
e_norm = teneva.accuracy(Y_full_real, Y_full_appr)
e_min = abs(y_min_real - np.min(Y_full_appr))
e_max = abs(y_max_real - np.max(Y_full_appr))

print(f'Error norm      : {e_norm:-7.1e}')
print(f'Error min value : {e_min:-7.1e}')
print(f'Error max value : {e_max:-7.1e}')

Error norm      : 3.9e-01
Error min value : 3.0e+03
Error max value : 3.4e+02


In [37]:
y_min = np.min(list(cache.values())) + y_max_ref
y_max = np.max(list(cache.values())) + y_max_ref

e_min = abs(y_min_real - y_min)
e_max = abs(y_max_real - y_max)

print(f'Error min value : {e_min:-7.1e}')
print(f'Error max value : {e_max:-7.1e}')

Error min value : 1.2e-12
Error max value : 0.0e+00


Вывод: как можно видеть, ни фига не работает.

In [86]:
def func(i):
    return Y_full_real[tuple(i)]

In [89]:
qttopt = QTTOpt(func, d, name, with_log=True)
qttopt.set_real(y_min=y_min_real)
qttopt.minimize(evals=1.E+4, rank=10)

Rosenbrock | evals: 2.0e+00+1.8e+01 | t: 3.9e-04 | swp:   0 (<-) | err val: 1.6e+03 | 
Rosenbrock | evals: 8.0e+00+1.3e+02 | t: 2.0e-03 | swp:   0 (<-) | err val: 1.4e+03 | 
Rosenbrock | evals: 2.6e+01+2.9e+02 | t: 4.1e-03 | swp:   0 (<-) | err val: 1.2e+03 | 
Rosenbrock | evals: 3.2e+01+1.0e+05 | t: 1.9e+00 | swp: 692 (->) | err val: 1.2e+03 | 


(array([1, 1, 1, 1, 0]), 1227.1780660107333)